<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

In [2]:
import settings
import local_settings
connection = psycopg2.connect(
    dbname=local_settings.DBNAME_PROJECT_SQL,
    user=local_settings.USER,
    host=local_settings.HOST,
    password=local_settings.PASSWORD,
    port=local_settings.PORT
)

In [6]:
def run_query(text_query):
    df = pd.read_sql(text_query, connection)
    display(df)
    
import warnings
warnings.filterwarnings('ignore')

# Юнит 3. Предварительный анализ данных

1. Посчитаем количество вакансий в нашей базе

In [9]:
run_query(f'''
    SELECT count(*) AS vacancies_count
    FROM vacancies
''')


,vacancies_count
0,49197


2. Посчитаем количество работодателей 

In [10]:
run_query(f'''
    SELECT count(*) AS employers_count
    FROM employers
''')

,employers_count
0,23501


3. Посчитаем количество регионов

In [11]:
run_query(f'''
    SELECT count(*) AS areas_count
    FROM areas
''')

,areas_count
0,1362


4. Посчитаем количество сфер деятельности

In [16]:
run_query(f'''
    SELECT count(*) AS industries_count
    FROM industries
''')

,industries_count
0,294


***

В выборке представлен широкий список вакансий (49197). Это и различные сферы деятельности (294), и различные регионы (1362), и большое количество работодателей (23501).

Некоторые работодатели открыли несколько вакансий.

# Юнит 4. Детальный анализ вакансий

1. Узнаем, сколько вакансий в каждом регионе

In [21]:
run_query(f'''
    SELECT a.name AS area, count(*) AS cnt
    FROM vacancies v
    LEFT JOIN areas a ON a.id = v.area_id
    GROUP BY a.name
    ORDER BY cnt DESC
    LIMIT 10
''')

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Определим, у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой

In [22]:
run_query(f'''
    SELECT count(*) AS vacancies_count
    FROM vacancies
    WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
''')

,vacancies_count
0,24073


3. Найдем средние значения для нижней и верхней границы зарплатной вилки

In [23]:

run_query(f'''
    SELECT 
        ROUND(AVG(salary_from), 2) as avg_salary_from,
        ROUND(AVG(salary_to), 2) AS avg_salary_to
    FROM vacancies
''')

,avg_salary_from,avg_salary_to
0,71064.66,110536.74


4. Выведем количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства, используемого в вакансиях


In [26]:
run_query(f'''
    SELECT schedule, employment, count(*) AS cnt
    FROM vacancies 
    GROUP BY schedule, employment
    ORDER BY cnt DESC
''')

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Узнаем количество вакансий в зависимости от требуемого опыта

In [27]:
run_query(f'''
    SELECT experience, count(*) AS cnt
    FROM vacancies 
    GROUP BY experience
    ORDER BY cnt
''')

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

1. Представлены не только вакансии в России, но и в других странах. Самое большое количество вакансий расположено в столицах и крупных городах. Лидируют Москва и Санкт-Петербург. На долю первых 6-ти крупных городов (Москва, Санкт-Петербург, Минск, Новосибирск, Алматы, Екатеринбург) приходится треть всех вакансий;

2. У половины вакансий не заполнена заработная плата. Вероятно, заработная плата работодателем назначается либо по результатам собеседования, либо озвучивается только лично кандидату;

3. В среднем работодатели указывают зарплатную вилку от 71 до 110 тысяч. Если это суммы до вычета НДФЛ, то на руки работники получат в среднем от 62 до 96. Скорее всего, в регионах суммы гораздо меньше, на среднее в большей степени влияют суммы вакансий в крупных городах (их подавляющее большинство);

4. БольшинстВ большинстве вакансий (72%) требуются сотрудники на полный день с полной занятостью. В 16% вакансий работодатели готовы принять сотрудников на удаленную работу на полный день, в 3% на гибкий график. Остальные комбинации графика раоты и типа трудоустройства гораздо менее популярны. Это закономерно, работодатели считают очную работу более производительной, нежели удаленную. И большая часть предприятий работают по стандартному графику 8 часов в день, 5 дней в неделю;

5. Больше половины вакансий рассчитаны на кандидатов с небольшим опытом (1-3 года). В 29% хотят видеть более опытных кандидатов (3-6 лет). Кандидатов без опыта готовы рассмотреть только в 15% вакансий, т.к. на подготовку таких работников требуется слишком много ресурсов компании. Слишком большой требуемый опыт работодатели, вероятнее всего, не указывают, чтобы не отпугнуть кандидатов и не уменьшать шансы найти нужного.

# Юнит 5. Анализ работодателей

1. Узнаем, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [67]:
run_query(f'''
    SELECT e.name AS employer_name, count(v.id) as vacancies_count
    FROM employers e
    LEFT JOIN vacancies v on e.id = v.employer_id
    GROUP BY e.name
    ORDER BY vacancies_count DESC
    LIMIT 10
    --LIMIT 1
    --LIMIT 1 OFFSET 4
''')   

,employer_name,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


2. Для каждого региона выведем количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдем тот, в котором наибольшее количество работодателей


In [59]:
run_query(f'''
    SELECT 
        a.name AS area_name, 
        sum(COALESCE(employers_count, 0)) AS employers_count, 
        sum(COALESCE(vacancies_count, 0)) AS vacancies_count
    FROM areas a
    LEFT JOIN (
        SELECT area AS area_id, count(distinct id) AS employers_count, 0 as vacancies_count
        FROM employers
        GROUP BY area
    
        UNION all
    
        SELECT area_id, 0, count(distinct id)
        FROM vacancies
        GROUP BY area_id) ev ON a.id = ev.area_id
    GROUP BY a.name
    ORDER BY employers_count DESC, vacancies_count DESC
    LIMIT 10
''')   

,area_name,employers_count,vacancies_count
0,Москва,5864.0,5333.0
1,Санкт-Петербург,2217.0,2851.0
2,Минск,1115.0,2112.0
3,Алматы,721.0,1892.0
4,Екатеринбург,609.0,1698.0
5,Новосибирск,573.0,2006.0
6,Казань,480.0,1415.0
7,Нижний Новгород,426.0,1670.0
8,Россия,410.0,0.0
9,Краснодар,409.0,1301.0


In [62]:
# Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей
# Можно отсортировать результат предыдущего запроса или сформировать запрос по-другому
run_query(f'''
    SELECT 
        a.name AS area_name, 
        count(e.id) AS employers_count
    FROM areas a
    INNER JOIN (
        SELECT id AS area_id 
        FROM areas
    
        EXCEPT
    
        SELECT area_id
        FROM vacancies) ua on ua.area_id = a.id
    LEFT JOIN employers e ON a.id = e.area
    GROUP BY a.name
    ORDER BY employers_count DESC
    LIMIT 10
''')   

,area_name,employers_count
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Ростовская область,18
5,Беларусь,18
6,Азербайджан,17
7,Нижегородская область,16
8,Республика Татарстан,16
9,Узбекистан,15


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [54]:
run_query(f'''
    SELECT e.name AS employer_name, count(distinct v.area_id) as areas_count
    FROM employers e
    LEFT JOIN vacancies v on e.id = v.employer_id
    GROUP BY e.name
    ORDER BY areas_count DESC
    LIMIT 10
''')   

,employer_name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Посчитаем количества работодателей, у которых не указана сфера деятельности

In [53]:
# Можно также левым соединением и условием на NULL
run_query(f'''
    SELECT count(*) 
    FROM (
        SELECT id
        FROM employers
        EXCEPT
        SELECT employer_id
        FROM employers_industries
    ) emp
''')   

,count
0,8419


5. Узнаем название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [69]:
run_query(f'''
    SELECT e.name as employer_name, count(ei.industry_id) as industry_count
    FROM employers_industries ei
    INNER JOIN employers e on ei.employer_id = e.id
    GROUP BY e.name
    HAVING count(ei.industry_id) = 4
    ORDER BY e.name
    LIMIT 1 OFFSET 2
''')   

,employer_name,industry_count
0,2ГИС,4


6. Выясним, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [52]:
run_query(f'''
    SELECT count(distinct ei.employer_id)
    FROM employers_industries ei
    INNER JOIN industries i on ei.industry_id = i.id
    WHERE i.name = 'Разработка программного обеспечения'
''')   

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [78]:
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
paragraphs = page.find('table', class_='sortable').find('tbody').find_all('td')
cities = [paragraph.find('a')['title'] for paragraph in paragraphs]
print(cities)
   
   # WITH x AS (SELECT id as emp_id FROM employers WHERE name = 'Яндекс')
   #  SELECT a.name, count(v.id) as vacancies_count
   #  FROM vacancies v
   #  INNER JOIN areas a ON v.area_id = a.id AND v.employer_id = (SELECT emp_id FROM x) AND a.name in
   #  ('Москва', 'Cанкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
   #  GROUP BY a.name
   #  ORDER BY vacancies_count DESC

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

1. Несомненным лидером по количеств вакансий является Яндекс (1933 вакансии, 8% всех вакансий). Далее следуют такие крупные компании, как Ростелеком, Тинькофф, СБЕР, Газпром нефть.

2. Наибольшее количество работодателей в Москве (25%). Далее следует Санкт-Петербург, Минск и Алматы. Примерно таких же лидеров мы видели и в топе регионов по количеству вакансий. Есть регионы, в которых представлены работодатели, но нет вакансий. В основном такое наблюдается в регионах "Москва" и "Казахстан". Вероятно, работодатели, которые имеют филиалы или размещают удаленные вакансии в нескольких регионах, указывают в профайле общий регион, а в вакансиях уже конкретный;

3. Вакансии в наибольшем количестве регионов представлены у таких крупных компаний, как Яндекс и Ростелеком. Также неожиданно в лидерах по количеству регионов оказались компания ООО "СпецремонтЭ и ИП Поляков Денис Иванович, которые занимаются ремонтом компьютеров и ищут соответствующих специалистов;

4. У 8419 работодателей (36%) не указана сфера деятельности. Это может исказить результаты анализа сфер деятельности работодателей;

6. Разработкой программного обеспечения занимаются 3553 работодателя (15% среди всех работодателей и 24% из указавших сферу деятельности). Развитие в этом направлении перспективно с точки зрения трудоустройства;


# Юнит 6. Предметный анализ

1. Посчитаем, сколько вакансий имеет отношение к данным (в её названии содержатся слова 'data' или 'данн')


In [28]:
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE LOWER(name) LIKE '%данн%' OR LOWER(name) LIKE '%data%'
''')    

,cnt
0,1771


2. Посчитаем, сколько есть подходящих вакансий для начинающего дата-сайентиста

In [47]:
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных| ml |machine learning|машинн%обучен)%'
        AND (LOWER(name) SIMILAR TO '%yunior%' OR experience = 'Нет опыта' OR employment = 'Стажировка')
''')    

,cnt
0,39


3. Узнаем, сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres

In [50]:
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных| ml |machine learning|машинн%обучен)%'
        AND LOWER(key_skills) SIMILAR TO '%(sql|postgres)%'
''')  

,cnt
0,180


4. Проверим, насколько популярен Python в требованиях работодателей к DS

In [72]:
run_query(f'''
    SELECT count(*) AS cnt
    FROM vacancies 
    WHERE name SIMILAR TO '%(data scientist|data science|исследователь данных|ML|machine learning|машинн%обучен)%'
        AND name NOT SIMILAR TO '%HTML%'
        AND LOWER(key_skills) SIMILAR TO '%python%'
''')  

,cnt
0,103


5. Проверим, сколько ключевых навыков в среднем указывают в вакансиях для DS

In [41]:
run_query(f'''
    SELECT ROUND(AVG(CHAR_LENGTH(key_skills)-CHAR_LENGTH(REPLACE(key_skills, E'\t',''))), 2) AS cnt
    FROM vacancies 
    WHERE LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных| ml |machine learning|машинн%обучен)%'
''')   

,cnt
0,5.41


6. Вычислим, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [44]:
run_query(f'''
    SELECT experience, ROUND(AVG((COALESCE(salary_from, 0) + COALESCE(salary_to, 0))/2), 2) AS avg_salary
    FROM vacancies 
    WHERE LOWER(name) SIMILAR TO '%(data scientist|data science|исследователь данных| ml |machine learning|машинн%обучен)%'
        AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    GROUP BY experience
    ORDER BY avg_salary DESC
''')      

,experience,avg_salary
0,От 3 до 6 лет,169103.54
1,От 1 года до 3 лет,114546.92
2,Нет опыта,56428.57


***

1. Найденно 1771 вакансий, предположительно связанных с данными. Сюда входят и специалисты DS, и аналитики, и специалисты по работе с базами данных. Это быстроразвивающееся направление, но пока такие вакансии составляют менее 4% всех вакансий;

2. У начинающих дата-сайентистов не так много возможностей. Найдено всего 26 таких вакансий. Видимо, работодатели не готовы/не хватает ресурсов на обучение начинающих специалистов;

3. SQL указан в качестве ключевого навыка в 180 вакансиях DS из 392 (46%). Соответственно специалисту DS обязательно нужно иметь хорошие навыки работы с запросами. А PostgreSQL - одна из самых распространенных баз данных, т.к. является бесплатной;

4. Знания Python требуются в 304 вакансиях DS из 392 (78%). Т.о. без знаний Python практически невозможно устроиться в DS;

5. В среднем работодатели указывают в вакансиях 5-6 ключевых навыков. Вероятно, такое количество навыков позволяет не отпугнуть потенциальных кандидатов;

6. Средняя заработная плата коррелирует с опытом кандидатов. Самая высокая средняя зарплата в DS у кандидатов с опытом более 3-х лет. Заработная плата кандидатов с опытом от 1 года выше, чем средняя заработная плата по всем вакансиям, которую мы рассчитывали ранее. Вероятно, это связано с тем, что эта сфера требует много профессиональных навыков и знаний.

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования